# Bibliotecas

In [87]:
import sys
from abc import ABC, abstractmethod
from collections import defaultdict

# Função Auxiliar

In [88]:
def _gerar_permutacoes(elementos):
    if len(elementos) <= 1:
        return [elementos]
    lista_de_todas_as_permutacoes = []
    for i in range(len(elementos)):
        primeiro_elemento = elementos[i]
        elementos_restantes = elementos[:i] + elementos[i+1:]
        permutacoes_do_resto = _gerar_permutacoes(elementos_restantes)
        for p in permutacoes_do_resto:
            lista_de_todas_as_permutacoes.append([primeiro_elemento] + p)
    return lista_de_todas_as_permutacoes

# Interface do Grafo

In [89]:
class Grafo(ABC):
    @abstractmethod
    def numero_de_vertices(self):
        pass

    @abstractmethod
    def numero_de_arestas(self):
        pass

    @abstractmethod
    def sequencia_de_graus(self):
        pass

    @abstractmethod
    def adicionar_aresta(self, u, v):
        pass

    @abstractmethod
    def remover_aresta(self, u, v):
        pass

    @abstractmethod
    def imprimir(self):
        pass
    
    #adicionando novos métodos
    @abstractmethod
    def is_simples(self):
        pass

    @abstractmethod
    def is_nulo(self):
        pass

    @abstractmethod
    def is_completo(self):
        pass

    # MÉTODOS DA ATIVIDADE em sala
    @abstractmethod
    def grau(self, vertice):
        pass

    @abstractmethod
    def tem_aresta(self, u, v):
        pass
    @abstractmethod
    def get_vertices(self):
        pass

    @abstractmethod
    def get_arestas(self):
        pass

    @abstractmethod
    def is_subgrafo(self, outro_grafo):
        pass

    @abstractmethod
    def is_subgrafo_gerador(self, outro_grafo):
        pass

    @abstractmethod
    def is_subgrafo_induzido(self, outro_grafo):
        pass
    @abstractmethod
    def is_isomorfo(self, outro_grafo):
        pass

# Classe GrafoDenso usando matriz de adjacência 

In [97]:
class GrafoDenso(Grafo):
    def __init__(self, vertices):
        if isinstance(vertices, int):
            self.rotulos = [str(i) for i in range(vertices)]
        else:
            self.rotulos = list(vertices)

        n = len(self.rotulos)
        self.matriz = [[0] * n for _ in range(n)]

    def numero_de_vertices(self):
        return len(self.rotulos)

    def numero_de_arestas(self):
        count = 0
        n = len(self.matriz)
        for i in range(n):
            for j in range(i, n):  # Inclui a diagonal para contar laços
                if i == j:
                    count += self.matriz[i][j] // 2 # Laços são contados uma vez
                else:
                    count += self.matriz[i][j]
        return count


    def sequencia_de_graus(self):
        graus = []
        n = len(self.matriz)
        for i in range(n):
        # Pegamos o rótulo correspondente ao índice da linha
            rotulo = self.rotulos[i]
            grau = sum(self.matriz[i])
        # Adicionamos a tupla (rótulo, grau) à lista
            graus.append((rotulo, grau))
        return graus
    
    def adicionar_aresta(self, u, v):
        i = self.rotulos.index(u)
        j = self.rotulos.index(v)
        self.matriz[i][j] += 1
        if i != j: # Evita adicionar duas vezes para laços
            self.matriz[j][i] += 1

    def remover_aresta(self, u, v):
        i = self.rotulos.index(u)
        j = self.rotulos.index(v)
        if self.matriz[i][j] > 0:
            self.matriz[i][j] -= 1
            if i != j:
                self.matriz[j][i] -= 1

    def imprimir(self):
        print("Matriz de Adjacência:")
        # Cabeçalho
        header = "   " + " ".join(f"{r:<2}" for r in self.rotulos)
        print(header)
        for i, rot in enumerate(self.rotulos):
            row_str = f"{rot:<2}" + " ".join(f"{val:<2}" for val in self.matriz[i])
            print(row_str)

    #novos métodos
    def is_simples(self):
        n = self.numero_de_vertices()
        for i in range(n):
            #verificar se possui laços, a diagonal principal deve ser 0
            if self.matriz[i][i]!= 0:
                return False
            #verificar arestas múltiplas
            for j in range(i,n):
                if self.matriz[i][j] > 1:
                    return False
        return True
    
    def is_nulo(self):
        # grafo nulo = não tem aresta --> todas as células da matriz são 0
        n = self.numero_de_vertices()
        for i in range(n):
            for j in range(n):
                if self.matriz[i][j] != 0:
                    return False
        return True
    
    def is_completo(self):
        n = self.numero_de_vertices()
        # 1. para ser completo tem que ser simples
        if not self.is_simples():
            return False
        #2. é considerado completo se houver 0 ou 1 vértice
        if n <= 1:
            return True
        #3. Para ser completo todos os elementos fora a  diagonal principal devem ser 1
        for i in range(n):
            for j in range(n):
                if i != j and self.matriz[i][j] != 1:
                    return False
        return True
    
    # --- IMPLEMENTAÇÃO DOS MÉTODOS DA ATIVIDADE em sala ---
    def get_vertices(self): return self.rotulos
    def get_arestas(self):
        arestas = []
        n = self.numero_de_vertices()
        for i in range(n):
            for j in range(i, n):
                for _ in range(self.matriz[i][j]):
                    arestas.append((self.rotulos[i], self.rotulos[j]))
        return arestas
    def grau(self, vertice):
        """Retorna o grau de um vértice específico."""
        # O grau de um vértice na matriz de adjacência é a soma de todos os elementos da sua linha.
        i = self.rotulos.index(vertice)
        return sum(self.matriz[i])

    def tem_aresta(self, u, v):
        """Verifica se existe uma aresta entre os vértices u e v."""
        # Para ter uma aresta, o valor na matriz correspondente deve ser maior que 0.
        i = self.rotulos.index(u)
        j = self.rotulos.index(v)
        return self.matriz[i][j] > 0


    def is_subgrafo(self, outro_grafo):
        # Etapa 1: Checagem de vértices (comum a qualquer implementação)
        vertices_self = self.get_vertices()
        if not set(vertices_self).issubset(set(outro_grafo.get_vertices())):
            return False

        # Etapa 2: Checagem de arestas usando a MATRIZ de adjacência
        copia_arestas_outro = list(outro_grafo.get_arestas())
        n = self.numero_de_vertices()
        
        # Iteramos pela metade superior da nossa própria matriz
        for i in range(n):
            for j in range(i, n):
                # O número de arestas entre i e j em 'self'
                num_arestas_self = self.matriz[i][j]
                
                # Se houver arestas, precisamos encontrá-las no outro grafo
                if num_arestas_self > 0:
                    aresta_procurada = tuple(sorted((self.rotulos[i], self.rotulos[j])))
                    
                    # Para cada aresta que temos, tentamos encontrar e remover uma correspondente
                    for _ in range(num_arestas_self):
                        encontrou = False
                        for k in range(len(copia_arestas_outro)):
                            if tuple(sorted(copia_arestas_outro[k])) == aresta_procurada:
                                copia_arestas_outro.pop(k)
                                encontrou = True
                                break
                        if not encontrou:
                            return False # Não encontrou uma aresta necessária
        return True

    def is_subgrafo_gerador(self, outro_grafo):
        # A lógica aqui é de alto nível e depende do is_subgrafo, que agora é específico
        if len(self.get_vertices()) != len(outro_grafo.get_vertices()):
            return False
        return self.is_subgrafo(outro_grafo)

    def is_subgrafo_induzido(self, outro_grafo):
        # Etapa 1: Checagem de vértices
        vertices_self = self.get_vertices()
        if not set(vertices_self).issubset(set(outro_grafo.get_vertices())):
            return False

        # Etapa 2: Contar arestas do outro grafo para consulta rápida
        contagem_arestas_outro = {}
        for aresta in outro_grafo.get_arestas():
            aresta_canon = tuple(sorted(aresta))
            contagem_arestas_outro[aresta_canon] = contagem_arestas_outro.get(aresta_canon, 0) + 1
        
        # Etapa 3: Checagem de indução usando a nossa MATRIZ
        n = self.numero_de_vertices()
        for i in range(n):
            for j in range(i, n):
                # Contagem de arestas em 'self' vem direto da matriz
                contagem_self = self.matriz[i][j]
                
                # Contagem de arestas em 'outro_grafo' vem do nosso mapa
                aresta_atual = tuple(sorted((self.rotulos[i], self.rotulos[j])))
                contagem_outro = contagem_arestas_outro.get(aresta_atual, 0)

                if contagem_self != contagem_outro:
                    return False
        return True
    def is_isomorfo(self, outro_grafo):
        if self.numero_de_vertices() != outro_grafo.numero_de_vertices(): return False
        if self.numero_de_arestas() != outro_grafo.numero_de_arestas(): return False
        seq_graus1 = sorted([g for _, g in self.sequencia_de_graus()])
        seq_graus2 = sorted([g for _, g in outro_grafo.sequencia_de_graus()])
        if seq_graus1 != seq_graus2: return False

        vertices1 = self.get_vertices()
        vertices2 = outro_grafo.get_vertices()
        permutacoes_de_vertices1 = _gerar_permutacoes(vertices1)

        for p in permutacoes_de_vertices1:
            #  CONSTRUÇÃO DO MAPEAMENTO 
            mapeamento = {}
            for i in range(len(vertices2)):
                mapeamento[vertices2[i]] = p[i]
           
            
            valido = True
            for i in range(len(vertices2)):
                for j in range(i, len(vertices2)):
                    u2, v2 = vertices2[i], vertices2[j]
                    u1_mapeado, v1_mapeado = mapeamento[u2], mapeamento[v2]
                    
                    if outro_grafo.tem_aresta(u2, v2) != self.tem_aresta(u1_mapeado, v1_mapeado):
                        valido = False; break
                if not valido: break
            
            if valido: return True
                
        return False


# Criando a classe GrafoEsparso usando lista de adjacências

In [ ]:
class GrafoEsparso(Grafo):
    def __init__(self, vertices):
        if isinstance(vertices, int):
            self.rotulos = [str(i) for i in range(vertices)]
        else:
            self.rotulos = list(vertices)

        self.lista = defaultdict(list)
        for rotulo in self.rotulos:
            self.lista[rotulo] = []

    def numero_de_vertices(self):
        return len(self.rotulos)
    
    def numero_de_arestas(self):
        count = 0
        for v in self.lista:
            count += len(self.lista[v])
        return count // 2
    
    def sequencia_de_graus(self):
        graus = []
        for v in self.rotulos:
            grau = len(self.lista[v])
            graus.append((v,grau))
        return graus
        
    def adicionar_aresta(self, u, v):
        if u in self.rotulos and v in self.rotulos:
            self.lista[u].append(v)
            if u != v:
                self.lista[v].append(u)
    
    def remover_aresta(self, u, v):
        if u in self.rotulos and v in self.rotulos:
            if v in self.lista[u]:
                self.lista[u].remove(v)
            if u != v and u in self.lista[v]:
                self.lista[v].remove(u)

    def imprimir(self):
        print("Lista de adjacência:")
        for v in self.rotulos:
            print(f"{v}: {self.lista[v]}")

    def is_simples(self):
        for v in self.lista:
            if v in self.lista[v]:
                return False
            if len(self.lista[v]) != len(set(self.lista[v])):
                return False
        return True
    
    def is_nulo(self):
        return self.numero_de_arestas() == 0
    
    def is_completo(self):
        if not self.is_simples():
            return False
        n = self.numero_de_vertices()
        if n <= 1:
            return True
        arestas_completas = n * (n-1) // 2
        return self.numero_de_arestas() == arestas_completas

    # MÉTODOS DA ATIVIDADE 3 ---
    # As implementações de get_vertices, is_subgrafo, is_subgrafo_gerador e is_subgrafo_induzido 
    def grau(self, vertice):
        """Retorna o grau de um vértice específico."""
        # O grau de um vértice na lista de adjacência é o número de vizinhos em sua lista.
        return len(self.lista[vertice])

    def tem_aresta(self, u, v):
        """Verifica se existe uma aresta entre os vértices u e v."""
        # Para ter uma aresta, v deve estar na lista de adjacência de u.
        return v in self.lista[u]

    def get_vertices(self): return self.rotulos
    def get_arestas(self):
        arestas, visitadas = [], set()
        for u in self.rotulos:
            for v in self.lista[u]:
                aresta_canonica = tuple(sorted((u, v)))
                if aresta_canonica not in visitadas:
                    contagem = self.lista[u].count(v)
                    for _ in range(contagem): arestas.append(aresta_canonica)
                    visitadas.add(aresta_canonica)
        return arestas
    
    def is_subgrafo(self, outro_grafo):
        """Verifica se este grafo (self) é um subgrafo de 'outro_grafo'."""
        # A lógica é exatamente a mesma da classe GrafoDenso.
        
        # ETAPA 1: VERIFICAR OS VÉRTICES
        vertices_self = self.get_vertices()
        vertices_outro = outro_grafo.get_vertices()
        for v_self in vertices_self:
            if v_self not in vertices_outro:
                return False

        # ETAPA 2: VERIFICAR AS ARESTAS 
        arestas_self = self.get_arestas()
        copia_arestas_outro = list(outro_grafo.get_arestas())

        for aresta_self in arestas_self:
            encontrou_aresta_correspondente = False
            aresta_self_ordenada = tuple(sorted(aresta_self))

            for i in range(len(copia_arestas_outro)):
                aresta_outro_ordenada = tuple(sorted(copia_arestas_outro[i]))
                if aresta_self_ordenada == aresta_outro_ordenada:
                    encontrou_aresta_correspondente = True
                    copia_arestas_outro.pop(i)
                    break 
            
            if not encontrou_aresta_correspondente:
                return False
        
        return True
    
    def is_subgrafo(self, outro_grafo):
        # Etapa 1: Checagem de vértices
        vertices_self = self.get_vertices()
        if not set(vertices_self).issubset(set(outro_grafo.get_vertices())):
            return False

        # Etapa 2: Checagem de arestas usando a LISTA de adjacência
        copia_arestas_outro = list(outro_grafo.get_arestas())
        visitados = set() # Para não processar a aresta A-B e B-A duas vezes

        for u in self.rotulos:
            for v in self.lista[u]:
                aresta_procurada = tuple(sorted((u, v)))
                if aresta_procurada in visitados:
                    continue # Já processamos essa aresta
                
                visitados.add(aresta_procurada)
                num_arestas_self = self.lista[u].count(v)

                for _ in range(num_arestas_self):
                    encontrou = False
                    for k in range(len(copia_arestas_outro)):
                        if tuple(sorted(copia_arestas_outro[k])) == aresta_procurada:
                            copia_arestas_outro.pop(k)
                            encontrou = True
                            break
                    if not encontrou:
                        return False
        return True

    def is_subgrafo_gerador(self, outro_grafo):
        if len(self.get_vertices()) != len(outro_grafo.get_vertices()):
            return False
        return self.is_subgrafo(outro_grafo)

    def is_subgrafo_induzido(self, outro_grafo):
        # Etapa 1: Checagem de vértices
        vertices_self = self.get_vertices()
        if not set(vertices_self).issubset(set(outro_grafo.get_vertices())):
            return False

        # Etapa 2: Contar arestas do outro grafo para consulta rápida
        contagem_arestas_outro = {}
        for aresta in outro_grafo.get_arestas():
            aresta_canon = tuple(sorted(aresta))
            contagem_arestas_outro[aresta_canon] = contagem_arestas_outro.get(aresta_canon, 0) + 1

        # Etapa 3: Checagem de indução usando a nossa LISTA de adjacência
        for u in vertices_self:
            for v in vertices_self:
                # Olhamos apenas para um lado (u <= v) para não checar duas vezes
                if u > v:
                    continue

                # Contagem de arestas em 'self' vem direto da lista
                contagem_self = self.lista[u].count(v)

                # Contagem de arestas em 'outro_grafo' vem do nosso mapa
                aresta_atual = tuple(sorted((u, v)))
                contagem_outro = contagem_arestas_outro.get(aresta_atual, 0)

                if contagem_self != contagem_outro:
                    return False
        return True
    
    def is_isomorfo(self, outro_grafo):
        if self.numero_de_vertices() != outro_grafo.numero_de_vertices(): return False
        if self.numero_de_arestas() != outro_grafo.numero_de_arestas(): return False
        seq_graus1 = sorted([g for _, g in self.sequencia_de_graus()])
        seq_graus2 = sorted([g for _, g in outro_grafo.sequencia_de_graus()])
        if seq_graus1 != seq_graus2: return False

        vertices1 = self.get_vertices()
        vertices2 = outro_grafo.get_vertices()
        permutacoes_de_vertices1 = _gerar_permutacoes(vertices1)
        arestas1_set = set(tuple(sorted(e)) for e in self.get_arestas())

        for p in permutacoes_de_vertices1:
            # CONSTRUÇÃO DO MAPEAMENTO
            mapeamento = {}
            for i in range(len(vertices2)):
                mapeamento[vertices2[i]] = p[i]
            # ------------------------
            
            valido = True
            for u2, v2 in outro_grafo.get_arestas():
                u1_mapeado, v1_mapeado = mapeamento[u2], mapeamento[v2]
                if tuple(sorted((u1_mapeado, v1_mapeado))) not in arestas1_set:
                    valido = False; break
            
            if valido: return True
                
        return False
    

# Testes de Isomorfismo

In [99]:
print("\n" + "="*30)
print("  TESTE DE ISOMORFISMO")
print("="*30)

G1 = GrafoEsparso(['A', 'B', 'C', 'D']); G1.adicionar_aresta('A', 'B'); G1.adicionar_aresta('B', 'C'); G1.adicionar_aresta('C', 'D'); G1.adicionar_aresta('D', 'A')
G2 = GrafoEsparso([1, 2, 3, 4]); G2.adicionar_aresta(1, 2); G2.adicionar_aresta(2, 3); G2.adicionar_aresta(3, 4); G2.adicionar_aresta(4, 1)
G3 = GrafoEsparso(['W', 'X', 'Y', 'Z']); G3.adicionar_aresta('W', 'X'); G3.adicionar_aresta('X', 'Y'); G3.adicionar_aresta('Y', 'Z')

print("Grafo G1 (Ciclo de 4 vértices):"); G1.imprimir()
print("\nGrafo G2 (Ciclo de 4 vértices com outros rótulos):"); G2.imprimir()
print("\nGrafo G3 (Caminho de 4 vértices):"); G3.imprimir()

print("\n--- Resultados ---")
print(f"G1 e G2 são isomorfos? {G1.is_isomorfo(G2)}")
print(f"G1 e G3 são isomorfos? {G1.is_isomorfo(G3)}")


  TESTE DE ISOMORFISMO
Grafo G1 (Ciclo de 4 vértices):
Lista de adjacência:
A: ['B', 'D']
B: ['A', 'C']
C: ['B', 'D']
D: ['C', 'A']

Grafo G2 (Ciclo de 4 vértices com outros rótulos):
Lista de adjacência:
1: [2, 4]
2: [1, 3]
3: [2, 4]
4: [3, 1]

Grafo G3 (Caminho de 4 vértices):
Lista de adjacência:
W: ['X']
X: ['W', 'Y']
Y: ['X', 'Z']
Z: ['Y']

--- Resultados ---
G1 e G2 são isomorfos? True
G1 e G3 são isomorfos? False


In [101]:
print("\n" + "="*30)
print("  TESTE DE ISOMORFISMO")
print("="*30)

G1 = GrafoEsparso(['A', 'B', 'C', 'D']); G1.adicionar_aresta('A', 'B'); G1.adicionar_aresta('B', 'C'); G1.adicionar_aresta('C', 'D'); G1.adicionar_aresta('D', 'A')
G2 = GrafoDenso([1, 2, 3, 4]); G2.adicionar_aresta(1, 2); G2.adicionar_aresta(2, 3); G2.adicionar_aresta(3, 4); G2.adicionar_aresta(4, 1)
G3 = GrafoEsparso(['W', 'X', 'Y', 'Z']); G3.adicionar_aresta('W', 'X'); G3.adicionar_aresta('X', 'Y'); G3.adicionar_aresta('Y', 'Z')

print("Grafo G1 (Esparso):"); G1.imprimir()
print("\nGrafo G2 (Denso):"); G2.imprimir()
print("\nGrafo G3 (Esparso):"); G3.imprimir()

print("\n--- Resultados ---")
print(f"G1 e G2 são isomorfos? {G1.is_isomorfo(G2)}")
print(f"G1 e G3 são isomorfos? {G1.is_isomorfo(G3)}")


  TESTE DE ISOMORFISMO
Grafo G1 (Esparso):
Lista de adjacência:
A: ['B', 'D']
B: ['A', 'C']
C: ['B', 'D']
D: ['C', 'A']

Grafo G2 (Denso):
Matriz de Adjacência:
   1  2  3  4 
1 0  1  0  1 
2 1  0  1  0 
3 0  1  0  1 
4 1  0  1  0 

Grafo G3 (Esparso):
Lista de adjacência:
W: ['X']
X: ['W', 'Y']
Y: ['X', 'Z']
Z: ['Y']

--- Resultados ---
G1 e G2 são isomorfos? True
G1 e G3 são isomorfos? False


# Teste de subgrafos usando GrafoDenso

In [45]:
print("\n" + "="*30)
print("  TESTE DE SUBGRAFOS (DENSO)")
print("="*30)

G = GrafoDenso(["A", "B", "C", "D"]); G.adicionar_aresta("A", "B"); G.adicionar_aresta("A", "C"); G.adicionar_aresta("B", "C"); G.adicionar_aresta("B", "D")
H1 = GrafoDenso(["A", "B", "D"]); H1.adicionar_aresta("A", "B"); H1.adicionar_aresta("B", "D")
H2 = GrafoDenso(["A", "B", "C", "D"]); H2.adicionar_aresta("A", "B"); H2.adicionar_aresta("B", "D")
H3 = GrafoDenso(["A", "B", "C"]); H3.adicionar_aresta("A", "B"); H3.adicionar_aresta("A", "C"); H3.adicionar_aresta("B", "C")
H4 = GrafoDenso(["A", "B", "C"]); H4.adicionar_aresta("A", "B"); H4.adicionar_aresta("B", "C")

print("\nGrafo G:"); G.imprimir()
print("\nGrafo H1:"); H1.imprimir()
print("\nGrafo H2:"); H2.imprimir()
print("\nGrafo H3:"); H3.imprimir()
print("\nGrafo H4:"); H4.imprimir()

print("\n--- Resultados ---")
print(f"H1 é subgrafo de G? {H1.is_subgrafo(G)}")
print(f"H2 é subgrafo de G? {H2.is_subgrafo(G)}")
print(f"H1 é subgrafo gerador de G? {H1.is_subgrafo_gerador(G)}")
print(f"H2 é subgrafo gerador de G? {H2.is_subgrafo_gerador(G)}")
print(f"H3 é subgrafo induzido de G? {H3.is_subgrafo_induzido(G)}")
print(f"H4 é subgrafo induzido de G? {H4.is_subgrafo_induzido(G)}")



  TESTE DE SUBGRAFOS (DENSO)

Grafo G:
Matriz de Adjacência:
   A  B  C  D 
A 0  1  1  0 
B 1  0  1  1 
C 1  1  0  0 
D 0  1  0  0 

Grafo H1:
Matriz de Adjacência:
   A  B  D 
A 0  1  0 
B 1  0  1 
D 0  1  0 

Grafo H2:
Matriz de Adjacência:
   A  B  C  D 
A 0  1  0  0 
B 1  0  0  1 
C 0  0  0  0 
D 0  1  0  0 

Grafo H3:
Matriz de Adjacência:
   A  B  C 
A 0  1  1 
B 1  0  1 
C 1  1  0 

Grafo H4:
Matriz de Adjacência:
   A  B  C 
A 0  1  0 
B 1  0  1 
C 0  1  0 

--- Resultados ---
H1 é subgrafo de G? True
H2 é subgrafo de G? True
H1 é subgrafo gerador de G? False
H2 é subgrafo gerador de G? True
H3 é subgrafo induzido de G? True
H4 é subgrafo induzido de G? False


# Testes de Subgrafo usando GrafoEsparso

In [54]:
print("\n" + "="*40)
print("    TESTE DE SUBGRAFOS (Esparso)")
print("="*40)

# Grafo G (será o grafo maior)
G = GrafoEsparso(["A", "B", "C", "D"])
G.adicionar_aresta("A", "B")
G.adicionar_aresta("A", "C")
G.adicionar_aresta("B", "C")
G.adicionar_aresta("B", "D")
print("\nGrafo G:")
G.imprimir()

# Subgrafo H1 (subgrafo simples de G)
H1 = GrafoEsparso(["A", "B", "D"])
H1.adicionar_aresta("A", "B")
H1.adicionar_aresta("B", "D")
print("\nGrafo H1:")
H1.imprimir()

# Subgrafo Gerador H2
H2 = GrafoEsparso(["A", "B", "C", "D"])
H2.adicionar_aresta("A", "B")
H2.adicionar_aresta("B", "D")
print("\nGrafo H2:")
H2.imprimir()

# Subgrafo Induzido H3 (induzido pelo conjunto de vértices {A,B,C})
H3 = GrafoEsparso(["A", "B", "C"])
H3.adicionar_aresta("A", "B")
H3.adicionar_aresta("A", "C")
H3.adicionar_aresta("B", "C")
print("\nGrafo H3:")
H3.imprimir()

# Grafo H4 (NÃO é subgrafo induzido por {A,B,C} pois falta a aresta A-C)
H4 = GrafoEsparso(["A", "B", "C"])
H4.adicionar_aresta("A", "B")
H4.adicionar_aresta("B", "C")
print("\nGrafo H4:")
H4.imprimir()

print("\n--- Resultados Corrigidos ---")
print(f"H1 é subgrafo de G? {H1.is_subgrafo(G)}")
print(f"H2 é subgrafo de G? {H2.is_subgrafo(G)}")
print(f"H1 é subgrafo gerador de G? {H1.is_subgrafo_gerador(G)}")
print(f"H2 é subgrafo gerador de G? {H2.is_subgrafo_gerador(G)}")
print(f"H3 é subgrafo induzido de G? {H3.is_subgrafo_induzido(G)}")
print(f"H4 é subgrafo induzido de G? {H4.is_subgrafo_induzido(G)}")


    TESTE DE SUBGRAFOS (Esparso)

Grafo G:
Lista de adjacência:
A: ['B', 'C']
B: ['A', 'C', 'D']
C: ['A', 'B']
D: ['B']

Grafo H1:
Lista de adjacência:
A: ['B']
B: ['A', 'D']
D: ['B']

Grafo H2:
Lista de adjacência:
A: ['B']
B: ['A', 'D']
C: []
D: ['B']

Grafo H3:
Lista de adjacência:
A: ['B', 'C']
B: ['A', 'C']
C: ['A', 'B']

Grafo H4:
Lista de adjacência:
A: ['B']
B: ['A', 'C']
C: ['B']

--- Resultados Corrigidos ---
H1 é subgrafo de G? True
H2 é subgrafo de G? True
H1 é subgrafo gerador de G? False
H2 é subgrafo gerador de G? True
H3 é subgrafo induzido de G? True
H4 é subgrafo induzido de G? False


# Teste Grafo Denso

In [50]:


g = GrafoDenso(["A", "B", "C", "D", "E"])  # cria grafo 

 # Adiciona arestas {(A,B), (A,C), (C,D), (C,E), (B,D)}
g.adicionar_aresta("A", "B")
g.adicionar_aresta("A", "C")
g.adicionar_aresta("C", "D")
g.adicionar_aresta("C", "E")
g.adicionar_aresta("B", "D")

    # Imprime informações
print("\n--- Grafo Inicial ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())

 # Remove a aresta (A,C)
g.remover_aresta("A", "C")

 # Imprime novamente
print("\n--- Após remover a aresta (A,C) ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())







--- Grafo Inicial ---
Matriz de Adjacência:
   A  B  C  D  E 
A 0  1  1  0  0 
B 1  0  0  1  0 
C 1  0  0  1  1 
D 0  1  1  0  0 
E 0  0  1  0  0 
Número de vértices: 5
Número de arestas: 5
Sequência de graus: [2, 2, 3, 2, 1]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False

--- Após remover a aresta (A,C) ---
Matriz de Adjacência:
   A  B  C  D  E 
A 0  1  0  0  0 
B 1  0  0  1  0 
C 0  0  0  1  1 
D 0  1  1  0  0 
E 0  0  1  0  0 
Número de vértices: 5
Número de arestas: 4
Sequência de graus: [1, 2, 2, 2, 1]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False


# Teste Grafo Esparso

In [49]:
# Instância com rótulos {A,B,C,D,E}
g = GrafoEsparso(["A", "B", "C", "D", "E"])

# Adiciona arestas {(A,B), (A,C), (C,D), (C,E), (B,D)}
g.adicionar_aresta("A", "B")
g.adicionar_aresta("A", "C")
g.adicionar_aresta("C", "D")
g.adicionar_aresta("C", "E")
g.adicionar_aresta("B", "D")

# Adiciona aresta duplicada (A,B) para testar multigrafo
g.adicionar_aresta("A", "B")

print("\n--- Grafo Inicial ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())

# Remove só uma instância da aresta (A,B)
g.remover_aresta("A", "B")

print("\n-- Após remover uma aresta (A,B) --")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())
print("O grafo é completo?", g.is_completo())




--- Grafo Inicial ---
Lista de adjacência:
A: ['B', 'C', 'B']
B: ['A', 'D', 'A']
C: ['A', 'D', 'E']
D: ['C', 'B']
E: ['C']
Número de vértices: 5
Número de arestas: 6
Sequência de graus: [('A', 3), ('B', 3), ('C', 3), ('D', 2), ('E', 1)]
O grafo é simples? False
O grafo é nulo? False
O grafo é completo? False

-- Após remover uma aresta (A,B) --
Lista de adjacência:
A: ['C', 'B']
B: ['D', 'A']
C: ['A', 'D', 'E']
D: ['C', 'B']
E: ['C']
Número de vértices: 5
Número de arestas: 5
Sequência de graus: [('A', 2), ('B', 2), ('C', 3), ('D', 2), ('E', 1)]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False
O grafo é completo? False
